In [39]:
# Intall dependencies
!pip install pandas boto3 openpyxl seaborn matplotlib

In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import io

# 1. Fetch
s3 = boto3.client('s3', endpoint_url='http://minio:9000', 
                  aws_access_key_id='minio', aws_secret_access_key='minioadmin')

print("Fetching 'telco_churn.xlsx'...")
obj = s3.get_object(Bucket='raw', Key='telco_churn.xlsx')
data = obj['Body'].read()

# 2. Verify the Magic Number (First 4 bytes of an Excel file should be PK..)
header = data[:4]
print(f"File Header Hex: {header.hex()}")
print(f"File Header Str: {header}")

Fetching 'telco_churn.xlsx'...
File Header Hex: 504b0304
File Header Str: b'PK\x03\x04'


In [41]:
df = pd.read_excel(io.BytesIO(data), engine='openpyxl')

In [42]:
%matplotlib inline
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [43]:
# 4. Quick Clean for EDA (Fixing TotalCharges)
# We force errors='coerce' to turn " " into NaN, then drop them for accurate plotting
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7049 non-null   object 
 1   gender            7049 non-null   object 
 2   SeniorCitizen     7049 non-null   int64  
 3   Partner           7049 non-null   object 
 4   Dependents        7049 non-null   object 
 5   tenure            7049 non-null   int64  
 6   PhoneService      7049 non-null   object 
 7   MultipleLines     7049 non-null   object 
 8   InternetService   7049 non-null   object 
 9   OnlineSecurity    7049 non-null   object 
 10  OnlineBackup      7049 non-null   object 
 11  DeviceProtection  7049 non-null   object 
 12  TechSupport       7049 non-null   object 
 13  StreamingTV       7049 non-null   object 
 14  StreamingMovies   7049 non-null   object 
 15  Contract          7049 non-null   object 
 16  PaperlessBilling  7049 non-null   object 


In [45]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7049.000000,7049.000000,7049.000000,7038.000000
mean,0.162009,32.366435,64.762151,2281.929731
std,0.368485,24.558613,30.084486,2266.541446
min,0.000000,0.000000,18.250000,-500.000000
25%,0.000000,9.000000,35.500000,400.500000
50%,0.000000,29.000000,70.350000,1397.100000
75%,0.000000,55.000000,89.850000,3791.000000
max,1.000000,72.000000,118.750000,8684.800000


In [46]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [37]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [63]:
df['PaymentMethod'].value_counts()

PaymentMethod
Electronic check             2366
Mailed check                 1614
Bank transfer (automatic)    1545
Credit card (automatic)      1524
Name: count, dtype: int64

In [75]:
df[~df['TotalCharges'].isna()][['tenure', 'MonthlyCharges', 'TotalCharges']]

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50
2,2,53.85,108.15
3,45,42.30,1840.75
4,2,70.70,151.65
...,...,...,...
7044,34,56.95,1889.50
7045,2,53.85,108.15
7046,38,69.50,2625.25
7047,67,102.95,-100.00


In [76]:
34 * 56.95

1936.3000000000002